# 🗿 SwapFace Image Colab Extended Edition V2
[![Открыть в Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/self-destruction/SwapFace/blob/main/SwapFace_Image_Colab_Edition_V2.ipynb)

##### С помощью этого блокнота ты можешь <b>в пару кликов</b> сделать дип-фейк фото. Для этого нужно загрузить главное фото и исходную фотографию лица, на которое нужно заменить лицо с главного фото. Для замены лица используется репозиторий [facefussion](https://github.com/facefusion/facefusion), но все возможные настройки перенесены в блокнот Google Colab.

In [ ]:
# @title # Загружаем главное фото
#@markdown ##### Предоставьте главное (.png, .jpg), <b>на котором</b> будут заменены лица.
#@markdown <font color=red>Здесь откроется проводник:</font>

from google.colab import files
import os
import matplotlib.pyplot as plt
from PIL import Image

main_img_folder = '/content/input/main' #@param {type:"string"}
!rm -rf {main_img_folder}
!mkdir -p {main_img_folder}
%cd {main_img_folder}
MAIN_IMAGE = ''

uploaded = files.upload()

for file_name in uploaded.keys():
    MAIN_IMAGE = os.path.join(main_img_folder, file_name)

print(f"{MAIN_IMAGE}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(MAIN_IMAGE))
plt.show()

In [ ]:
# @title # Загружаем исходное лицо
#@markdown ##### Предоставьте изображение лица (.png, .jpg), <b>на которое</b> необходимо заменить лицо
#@markdown <font color=red>Здесь откроется проводник:</font>

from google.colab import files
import os
import matplotlib.pyplot as plt
from PIL import Image

img_folder = '/content/input/source' #@param {type:"string"}
!rm -rf {img_folder}
!mkdir -p {img_folder}
%cd {img_folder}
SOURCE_FACE_IMAGE = ''
TARGET_FACE_IMAGES = []

uploaded = files.upload()

for file_name in uploaded.keys():
    SOURCE_FACE_IMAGE = os.path.join(img_folder, file_name)

print(f"{SOURCE_FACE_IMAGE}")
plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(SOURCE_FACE_IMAGE))
plt.show()

In [ ]:
# @title #Установка facefussion
import ipywidgets as widgets
import os
from pathlib import PosixPath
import shutil

ROOT_DIR = '/content'
!mkdir -p {ROOT_DIR}/output

%cd {ROOT_DIR}
!git clone https://github.com/facefusion/facefusion -b next
SWAPFACE_REPO_PATH = f'{ROOT_DIR}/facefusion'
%cd {SWAPFACE_REPO_PATH}

!pip install -q moviepy yt_dlp ffmpeg ffmpeg-python virtualenv
!virtualenv venv
!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python install.py --torch cuda --onnxruntime cuda

!sed -i 's/def analyse_frame(frame : Frame) -> bool:/def analyse_frame(frame : Frame) -> bool: True\ndef analyse_frame123(frame : Frame) -> bool:/g' {SWAPFACE_REPO_PATH}/facefusion/content_analyser.py

%cd {ROOT_DIR}
print('Готово! Продолжай дальше!')

In [ ]:
from PIL import Image
import os
from pathlib import Path, PurePath
import subprocess

debug_mode = True

# @title # Замена
# @markdown ### <b>Настройки дип-фейка:</b>
# @markdown ##### Улучшение лица после замены:
face_enhancer_model = 'none' # @param ["none", "codeformer", "gfpgan_1.2", "gfpgan_1.3", "gfpgan_1.4", "gpen_bfr_512", "restoreformer"]
face_enhancer_blend = 100 # @param {type:"slider", min:0, max:100, step:1}
face_enhancer_blend = str(face_enhancer_blend)
# @markdown ##### Модель замены лица (<font color="grey">inswapper_128</font> - обычная, <font color="grey">inswapper_128_fp16</font> - быстрая, <font color="grey">simswap</font> - экспериментальные модели более выского разрешения, обученные любителями):
face_swapper_model = 'inswapper_128_fp16' # @param ["inswapper_128", "inswapper_128_fp16", "simswap_256", "simswap_512_unofficial"]
face_detector_size = '640x640' # @param ["320x320", "480x480", "512x512", "640x640", "768x768", "1024x1024"]
# @markdown ##### Улучшение финального кадра:
frame_enhancer_model = 'real_esrnet_x4plus' # @param ["none", "real_esrgan_x2plus", "real_esrgan_x4plus", "real_esrnet_x4plus"]
frame_enhancer_blend = 100 # @param {type:"slider", min:0, max:100, step:1}
frame_enhancer_blend = str(frame_enhancer_blend)
# @markdown ##### Тип замены (reference - лицо с текущего выбранного кадра, many - несколько лиц):
face_selector_mode = 'reference' # @param ["reference", "many"]
# @markdown ##### Условия выбора лица (только если выбран тип замены "many"):
face_analyser_gender = 'none' # @param ["none", "male", "female"]
if face_analyser_gender != 'none':
  face_analyser_gender = f"--face-analyser-gender {face_analyser_gender} "
else:
  face_analyser_gender = ''
face_analyser_age = 'none' # @param ["none", "child", "teen", "adult", "senior"]
if face_analyser_age != 'none':
  face_analyser_age = f"--face-analyser-age {face_analyser_age} "
else:
  face_analyser_age = ''
face_detector_model = 'retinaface' # @param ["retinaface", "yunet"]

OUTPUT_PATH = f'{ROOT_DIR}/output'
output_name = PurePath(MAIN_IMAGE).stem
original_extension = PurePath(MAIN_IMAGE).suffix
processed_file = os.path.join(OUTPUT_PATH, output_name + original_extension)

%cd {SWAPFACE_REPO_PATH}

plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(Path(MAIN_IMAGE)))
plt.show()

# ---


face_swapper_processor = 'face_swapper'
face_enhancer_processor = 'face_enhancer'
frame_enhancer_processor = 'frame_enhancer'

frame_processors = face_swapper_processor

if face_enhancer_model != 'none':
    frame_processors = frame_processors + ' ' + face_enhancer_processor
else:
    face_enhancer_model = 'codeformer'
if frame_enhancer_model != 'none':
    frame_processors = frame_processors + ' ' + frame_enhancer_processor
else:
    frame_enhancer_model = 'real_esrgan_x2plus'

cmd = (
    f"--source {Path(SOURCE_FACE_IMAGE)} "
    f"--target {Path(MAIN_IMAGE)} "
    f"--output {processed_file} "
    f"--headless "
    f"--execution-providers cuda "
    f"--execution-thread-count 128 "
    f"--execution-queue-count 16 "
    f"--max-memory 128 "
    f"--face-analyser-order left-right "
    f"{face_analyser_age}"
    f"{face_analyser_gender}"
    f"--face-detector-model {face_detector_model} "
    f"--face-detector-size {face_detector_size} "
    f"--face-detector-score 0.5 "
    f"--face-selector-mode {face_selector_mode} "
    f"--reference-face-position 0 "
    f"--reference-face-distance 1.5 "
    f"--reference-frame-number 0 "
    f"--trim-frame-start 0 "
    f"--trim-frame-end 0 "
    f"--temp-frame-format jpg "
    f"--temp-frame-quality 100 "
    f"--output-image-quality 100 "
    f"--output-video-encoder libx264 "
    f"--output-video-quality 100 "
    f"--keep-fps "
    f"--output-image-quality 100 "
    f"--frame-processors {frame_processors} "
    f"--face-enhancer-model {face_enhancer_model} "
    f"--face-enhancer-blend {face_enhancer_blend} "
    f"--face-swapper-model {face_swapper_model} "
    f"--frame-enhancer-model {frame_enhancer_model} "
    f"--frame-enhancer-blend {frame_enhancer_blend} "
    f"--ui-layouts default"
)

if debug_mode == True:
    print(cmd)

!source {SWAPFACE_REPO_PATH}/venv/bin/activate; python run.py {cmd}

plt.figure(figsize=(7, 5))
plt.axis('off')
plt.imshow(Image.open(processed_file))
plt.show()